In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
print(keras.__version__)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from math import nan
from keras.callbacks import ModelCheckpoint

!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.layers import CRF

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pkg_resources

packages = ['numpy', 'pandas', 'keras', 'keras-contrib', 'tensorflow']
# Add more packages as needed

for package in packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package.capitalize()} version: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package.capitalize()} is not installed.")



## Importing the dataset for named entity recognition model

In [ ]:
dframe = pd.read_csv("../input/entity-annotated-corpus/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)

In [ ]:
dframe

## Data preprocessing

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
dframe.columns

## We want word, pos, sentence_idx and tag as an input 

In [ ]:
dataset=dframe.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word',"pos"],axis=1)

In [ ]:
dataset.info()

In [ ]:
!pip install gensim

In [ ]:
import gensim

print("Gensim version:", gensim.__version__)


In [ ]:
dim = 512

In [ ]:
import pandas as pd

df =  pd.read_parquet('https://github.com/alwanrahmanas/ner/blob/main/labeled/372%20clean%20tanpa%20makalah%20sidang%20blablabla.parquet?raw=true')

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
df.head()

In [ ]:
if 'document_id' in df.columns:
    df.drop('document_id',axis=1,inplace=True)

In [ ]:
# Ensure all tokens are strings
def clean_tokens(token_list):
    return [str(token) for token in token_list]

df['tokens'] = df['tokens'].apply(clean_tokens)

In [ ]:
# Separate tokens and tags
tokens = df['tokens'].tolist()
tags = df['ner_tags'].tolist()

In [ ]:
# Flatten the list of tokens for training Word2Vec
flat_tokens = [token for sublist in tokens for token in sublist]

In [ ]:
from gensim.models import Word2Vec
# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=tokens, size=300, window=5, min_count=1, workers=4, sg=1)



In [ ]:
# Access the vocabulary (index to word mapping)
vocab = word2vec_model.wv.vocab

# Create a word-to-index mapping
word_index = {word: vocab[word].index for word in vocab}

# Calculate the vocabulary size
vocab_size = len(vocab)

In [ ]:
n_words = len(word_index); n_words

In [ ]:
# Convert tokens to sequences of word indices
sequences = [[word_index[word] for word in sentence] for sentence in tokens]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder for the tags
tag_encoder = LabelEncoder()
tag_encoder.fit([tag for tag_list in tags for tag in tag_list])
tag_index = {tag: idx for idx, tag in enumerate(tag_encoder.classes_)}
num_tags = len(tag_index)

# Convert tags to sequences of indices
tag_sequences = [[tag_index[tag] for tag in tag_list] for tag_list in tags]


In [ ]:
# Ensure all sequences are padded to the same length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences=sequences, maxlen=512, padding='post', value=tag_index['O'])
padded_tag_sequences = pad_sequences(sequences=tag_sequences, maxlen=512, padding='post', value=tag_index['O'])

print("Vocabulary Size:", vocab_size)
print("Tag Index:", tag_index)
print("Padded Sequences:\n", padded_sequences)
print("Padded Tag Sequences:\n", padded_tag_sequences)


In [ ]:
# ### s = set()
# for filename in padded_tag_sequences[2]:
#     s.add(filename)
# print(s)
tag_index

In [ ]:
# Define X and y variables
from keras.utils import to_categorical

X = padded_sequences
y = [to_categorical(i, num_classes=7) for i in padded_tag_sequences]

In [ ]:
# Split the data into train and test sets
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

## Create list of list of tuples to differentiate each sentence from each other

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                            s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence #").apply(agg_func)

        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped.get("Sentence: {}".format(self.n_sent), None)
            if s is not None:
                self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(dataset)

In [ ]:
sentences = getter.sentences

In [ ]:
print(sentences[0])

In [ ]:
maxlen = max([len(s) for s in sentences])
print ('Maximum sequence length:', maxlen)

In [ ]:
# Check how long sentences are so that we can pad them
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

In [ ]:
plt.hist([len(s) for s in sentences], bins=50)
plt.show()

In [ ]:
dataset["Word"]

In [ ]:
words = list(set(dataset["Word"].values))
words.append("ENDPAD")

In [ ]:
n_words = len(words); n_words

## Fix the tags

In [ ]:
dataset.head()

In [ ]:
tags = []
for tag in set(dataset["Tag"].values):
    if tag is nan or isinstance(tag, float):
        tags.append('unk')
    else:
        tags.append(tag)
print(tags)

In [ ]:
n_tags = len(tags); n_tags

**Converting words to numbers and numbers to words**

In [ ]:
from future.utils import iteritems
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [ ]:
word2idx['nlp']

In [ ]:
tag2idx["B-TUJUAN"]

In [ ]:
idx2tag[5]

In [ ]:
idx2tag

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [ ]:
word2idx

In [ ]:
np.array(X).shape

In [ ]:
X

In [ ]:
X = pad_sequences(maxlen=dim, sequences=X, padding="post",value=n_words - 1)

In [ ]:
X

In [ ]:
y_idx = [[tag2idx[w[1]] for w in s] for s in sentences]
print(sentences[100])
print(y_idx[100])

In [ ]:
tag2idx

In [ ]:
y = pad_sequences(maxlen=dim, sequences=y_idx, padding="post", value=tag2idx["O"])
print(y_idx[100])

In [ ]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

## Import Keras

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k

## Keras version

In [ ]:
print(k.__version__)

## Model
 **Pay attention to the word embedding size

input = Input(shape=(dim,))
word_embedding_size = 300
model = Embedding(input_dim=n_words, output_dim=word_embedding_size, input_length=dim)(input)
model = Bidirectional(LSTM(units=word_embedding_size, 
                           return_sequences=True, 
                           dropout=0.05, 
                           recurrent_dropout=0.05, 
                           kernel_initializer=k.initializers.he_normal()))(model)
model = LSTM(units=word_embedding_size * 2, 
             return_sequences=True, 
             dropout=0.5, 
             recurrent_dropout=0.5, 
             kernel_initializer=k.initializers.he_normal())(model)
model = TimeDistributed(Dense(n_tags, activation="relu"))(model)  # previously softmax output layer

crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [ ]:
import keras.backend as K

# Define custom loss functions
def focal_loss(y_true, y_pred, gamma=2.0):
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    return -K.sum((1 - y_pred) ** gamma * y_true * K.log(y_pred), axis=-1)

def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return -(2.0 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Define custom metric function
def custom_metric(y_true, y_pred, method='focal'):
    # You can choose either Focal Loss or Dice Loss here
    if method == 'focal':
        loss = focal_loss(y_true, y_pred)
    elif method == 'dice':
        loss = dice_loss(y_true, y_pred)
    return loss



In [ ]:
class CustomCRF(CRF):
    def call(self, inputs, mask=None, training=None):
        sequence_lengths = K.sum(K.cast(mask, 'int32'), axis=-1)
        self.sequence_lengths = sequence_lengths  # Store it for later use
        return super(CustomCRF, self).call(inputs, mask=mask, training=training)


In [ ]:
n_tags

In [ ]:
# Define your model
# Define your model
input = Input(shape=(dim,))
word_embedding_size = 300
model = Embedding(input_dim=n_words, output_dim=word_embedding_size, input_length=dim)(input)
model = Bidirectional(LSTM(units=word_embedding_size, 
                           return_sequences=True, 
                           dropout=0.05, 
                           recurrent_dropout=0.05, 
                           kernel_initializer='he_normal'))(model)
model = LSTM(units=word_embedding_size * 2, 
             return_sequences=True, 
             dropout=0.5, 
             recurrent_dropout=0.5, 
             kernel_initializer='he_normal')(model)
output = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # Softmax output layer




In [ ]:
# Define your model
model = Model(input, output)

In [ ]:
adam = k.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
# #model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])
# # Compile the model with custom loss
# model = Model(input, out)
# Compile the model with custom loss and accuracy metric
# Compile the model with custom loss and custom metric
model.compile(optimizer=adam, loss=custom_metric, metrics=[custom_metric])


In [ ]:
model.summary()

model

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
# Assuming 'model' is your Keras model
plot_model(model, to_file='model_visualization.png', show_shapes=True, show_layer_names=True)

## Save the model after each epoch if validation is better

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

# Define a custom callback to calculate validation accuracy
class ValidationAccuracy(Callback):
    def __init__(self):
        super(ValidationAccuracy, self).__init__()
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        val_loss = logs.get('val_loss')
        if val_loss is None:
            return
        val_acc = 1 - val_loss  # Assuming higher val_loss means lower accuracy (change this if it's the opposite)
        logs['val_acc'] = val_acc

# Define the static file path for saving the best model
filepath = "best_model.hdf5"

# Create the custom callback instance
val_accuracy_callback = ValidationAccuracy()

# Create the ModelCheckpoint callback with a static file name
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_acc', 
                               patience=5, 
                               verbose=1, 
                               mode='max', 
                               restore_best_weights=True)

# Use the callbacks in the training loop
callbacks_list = [val_accuracy_callback, checkpoint, early_stopping]

# Example of model fitting with the callbacks
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, 
#                     batch_size=64, callbacks=callbacks_list)


In [ ]:
# from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

# # Define a custom callback to calculate validation accuracy
# class ValidationAccuracy(Callback):
#     def __init__(self, filepath):
#         super(ValidationAccuracy, self).__init__()
#         self.filepath = filepath
        
#     def on_epoch_end(self, epoch, logs=None):
#         if logs is None:
#             logs = {}
#         val_loss = logs.get('val_loss')
#         if val_loss is None:
#             return
#         val_acc = 1 - val_loss  # Assuming higher val_loss means lower accuracy (change this if it's the opposite)
#         logs['val_acc'] = val_acc

# # Define the file path with the dynamic name
# filepath = "ner-bi-lstm-td-model-{epoch:02d}-{val_acc:.2f}.hdf5"

# # Create the custom callback instance
# val_accuracy_callback = ValidationAccuracy(filepath)

# # Create the ModelCheckpoint callback
# checkpoint = ModelCheckpoint(filepath, 
#                              monitor='val_acc', 
#                              verbose=1, 
#                              save_best_only=True, 
#                              mode='max')
# # Create the EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_acc', 
#                                patience=20, 
#                                verbose=1, 
#                                mode='max', 
#                                restore_best_weights=True)

# # Use the callbacks in the training loop
# callbacks_list = [val_accuracy_callback, checkpoint, early_stopping]


In [ ]:
# <!-- from sklearn.model_selection import KFold

# # Define the number of folds for cross-validation
# n_splits = 3  # for example, you can adjust this number as needed

# # Initialize KFold cross-validator
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Initialize lists to store training/validation histories
# training_histories = []
# validation_histories = []

# # Convert y_train to a NumPy array if it's not already one
# y_train_array = np.array(y_train)

# # Perform cross-validation
# for train_index, val_index in kf.split(X_train):
#     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train_array[train_index], y_train_array[val_index] -->

#     # Fit the model on the current fold
#     history = model.fit(X_train_fold, np.array(y_train_fold), batch_size=16, epochs=5, 
#                         validation_data=(X_val_fold, np.array(y_val_fold)), verbose=1, 
#                         callbacks=callbacks_list)
    
#     # Store training/validation history
#     training_histories.append(history.history['crf_viterbi_accuracy'])  # Use 'crf_viterbi_accuracy' instead of 'accuracy'
#     validation_histories.append(history.history['val_crf_viterbi_accuracy'])  # Use 'val_crf_viterbi_accuracy' instead of 'val_accuracy'



In [ ]:
# # Assuming X_train is your tokenized and padded input data
# print("Shape of X_train:", X_train.shape)

# # Assuming n_tags is the number of unique tags
# print("Number of unique tags (classes):", n_tags)

# # Confirm the last dimension of X_train matches the number of classes
# assert X_train.shape[-1] == n_tags, "Mismatch between the last dimension of X_train and the number of classes (n_tags)"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

# Define the number of folds for cross-validation
n_splits = 7  # Adjust this number as needed

# Convert y_train to a NumPy array if it's not already one
y_train_array = np.array(y_train)

# Initialize KFold cross-validator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store training/validation histories
training_histories = []
validation_histories = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train_array[train_index], y_train_array[val_index]

    # Fit the model on the current fold
    history = model.fit(X_train_fold, np.array(y_train_fold), batch_size=64, epochs=100, 
                        validation_data=(X_val_fold, np.array(y_val_fold)), verbose=1, 
                        callbacks=callbacks_list)
    
    # Store training/validation history
    training_histories.append(history.history['custom_metric'])  # Use the name of your custom metric function
    validation_histories.append(history.history['val_custom_metric'])  # Use the name of your custom metric function


plt.show()

# batch size 32, k 9

In [ ]:
# Assuming callbacks_list contains ModelCheckpoint callback at index 1
model_checkpoint = callbacks_list[1]

# Access the model from the ModelCheckpoint object
model = model_checkpoint.model

# Save the model using tf.keras.Model.save
model.save(f"{filepath} Word2Vec 512 64 100 model_checkpoint.h5")


## Accumulate metrics by tag 

In [ ]:
# Plotting the loss for each fold
plt.figure(figsize=(12, 8))

# Plot training loss for each fold
for i, training_loss in enumerate(training_histories):
    plt.plot(training_loss, label=f'Training Fold {i+1}')
    
# Plot validation loss for each fold
for i, validation_loss in enumerate(validation_histories):
    plt.plot(validation_loss, label=f'Validation Fold {i+1}', linestyle='--')

plt.title('Training and Validation Loss for Each Fold')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

# Assuming training_histories and validation_histories are lists of lists
# where each inner list contains the loss values for each epoch of a fold

# Initialize lists to hold the final epoch loss values
final_training_losses = []
final_validation_losses = []

# Extract the last epoch loss for each fold
for training_loss, validation_loss in zip(training_histories, validation_histories):
    final_training_losses.append(training_loss[-1])
    final_validation_losses.append(validation_loss[-1])

# Plotting the final epoch loss for each fold
plt.figure(figsize=(12, 8))

# Plot final training loss for each fold
plt.plot(final_training_losses, label='Final Training Loss', marker='o')

# Plot final validation loss for each fold
plt.plot(final_validation_losses, label='Final Validation Loss', marker='o', linestyle='--')

plt.title('Final Epoch Training and Validation Loss for Each Fold')
plt.xlabel('Fold')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:

# Save the plot to a file
plt.savefig('final_epoch_loss_BUS 75 epoch 512 dim 64 batch size k 5 on nSCRD with word2vec plot.png')  # Save as a PNG file

In [ ]:
from keras.models import load_model

# Load the model
best_model = load_model("/kaggle/working/nsCRD 512 64 100 model_checkpoint.h5", custom_objects={'custom_metric': custom_metric})


In [ ]:
TP = {}
TN = {}
FP = {}
FN = {}
for tag in tag2idx.keys():
    TP[tag] = 0
    TN[tag] = 0    
    FP[tag] = 0    
    FN[tag] = 0    

def accumulate_score_by_tag(gt, pred):
    """
    For each tag keep stats
    """
    if gt == pred:
        TP[gt] += 1
    elif gt != 'O' and pred == 'O':
        FN[gt] +=1
    elif gt == 'O' and pred != 'O':
        FP[gt] += 1
    else:
        TN[gt] += 1


In [ ]:
TP

## Single prediction and verbose results

In [ ]:
np.array([X_test[i]])

In [ ]:
i = 9
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
gt = np.argmax(y_test[i], axis=-1)
print("{:14}: ({:5}): {}".format("Word", "True", "Pred"))
# for idx, (w,pred) in enumerate(zip(X_test[i],p[0])):
#     #
#     print("{:14}: ({:5}): {}".format(words[w],idx2tag[gt[idx]],tags[pred]))

## Predict everything at once

In [ ]:
p = model.predict(np.array(X_test))  

In [ ]:
X_test

## The output is 3d: sent x word x tag prob (softmax)

In [ ]:
p.shape

## Standard Classification Report

In [ ]:
from sklearn.metrics import classification_report

Grab the 3d dimension and return the index of the highest probability ... the index matches the tag value
np.argmax(p, axis=2)

In [ ]:
np.argmax(p, axis=2)[0]

In [ ]:
report=classification_report(np.argmax(y_test, 2).ravel(), np.argmax(p, axis=2).ravel(),labels=list(idx2tag.keys()), target_names=list(idx2tag.values()))
print(report)

focal loss sentence level resampling 512 64 epoch 75 k 5
    B-METODE       0.29      0.21      0.24       511
    B-TEMUAN       0.10      0.07      0.08       473
    I-METODE       0.17      0.07      0.10       233
           O       0.63      0.46      0.53      1275
    I-TUJUAN       0.48      0.33      0.39     11482
    B-TUJUAN       0.66      0.25      0.36      3836
    I-TEMUAN       0.89      0.95      0.92    100462

    accuracy                           0.86    118272
   macro avg       0.46      0.33      0.38    118272
weighted avg       0.84      0.86      0.84    118272

BUS 512 64 k=7 
precision    recall  f1-score   support

    B-METODE       0.00      0.00      0.00       171
    B-TEMUAN       0.06      0.01      0.02       152
    I-METODE       0.25      0.01      0.03        76
           O       0.47      0.41      0.44       410
    I-TUJUAN       0.67      0.67      0.67      3582
    B-TUJUAN       0.72      0.36      0.48      1204
    I-TEMUAN       0.94      0.96      0.95     32805

    accuracy                           0.90     38400
   macro avg       0.44      0.35      0.37     38400
weighted avg       0.89      0.90      0.89     38400

focal loss 512 32 k 9 
precision    recall  f1-score   support

    B-TUJUAN       0.84      0.72      0.77       288
    B-TEMUAN       0.77      0.66      0.71       620
    B-METODE       0.68      0.38      0.49       671
    I-METODE       0.89      0.88      0.88      1839
    I-TEMUAN       0.99      0.99      0.99     14484
           O       0.99      1.00      1.00    135312
    I-TUJUAN       0.98      0.98      0.98      4482

    accuracy                           0.99    157696
   macro avg       0.88      0.80      0.83    157696
weighted avg       0.99      0.99      0.99    157696


focal loss 256, 64, 50

              precision    recall  f1-score   support

    B-TUJUAN       0.73      0.68      0.70       197
    I-TUJUAN       0.97      0.99      0.98      3065
    I-TEMUAN       0.98      0.99      0.99     10834
    B-METODE       0.69      0.21      0.32       498
    I-METODE       0.87      0.85      0.86      1354
    B-TEMUAN       0.70      0.51      0.59       453
           O       0.99      0.99      0.99     48111

    accuracy                           0.98     64512
   macro avg       0.85      0.74      0.78     64512
weighted avg       0.98      0.98      0.98     64512

 focal loss 2 jam dim 512, epoch 100, batch size 64, k 7 abscon
 precision    recall  f1-score   support

    B-TUJUAN       0.85      0.85      0.85       239
    I-TUJUAN       0.99      0.99      0.99      3789
    I-TEMUAN       0.97      0.97      0.97     11325
    B-METODE       0.73      0.53      0.61       555
    I-METODE       0.91      0.91      0.91      1430
    B-TEMUAN       0.87      0.74      0.80       471
           O       0.99      1.00      0.99    111215

    accuracy                           0.99    129024
   macro avg       0.90      0.85      0.87    129024
weighted avg       0.99      0.99      0.99    129024

In [ ]:
# import numpy as np

# prec = [0.69 , 0.0, 0.00, 0, 0.15, 0.08, 0.00]
# rec = [0.97, 0, 0.00, 0, 0.0, 0.02, 0]
# f1 = [0.8, 0.0, 0.00, 0, 0.01, 0.03, 0]

# av_prec = np.mean(prec)
# av_rec = np.mean(rec)
# av_f1 = np.mean(f1)


In [ ]:
# print("Average Precision:", av_prec)
# print("Average Recall:", av_rec)
# print("Average F1-score:", av_f1)


## Accumulate the scores by tag

In [ ]:
for i, sentence in enumerate(X_test):
    y_hat = np.argmax(p[i], axis=-1)
    gt = np.argmax(y_test[i], axis=-1)
    for idx, (w,pred) in enumerate(zip(sentence,y_hat)):
        accumulate_score_by_tag(idx2tag[gt[idx]],tags[pred])

## How did Classification perform for each tag

In [ ]:
for tag in tag2idx.keys():
    print(f'tag:{tag}')    
    print('\t TN:{:10}\tFP:{:10}'.format(TN[tag],FP[tag]))
    print('\t FN:{:10}\tTP:{:10}'.format(FN[tag],TP[tag]))    

In [ ]:
# Summing all values in the dictionaries
total_TP = sum(TP.values())
total_TN = sum(TN.values())
total_FP = sum(FP.values())
total_FN = sum(FN.values())

# Printing the totals
print("Total TP:", total_TP)
print("Total TN:", total_TN)
print("Total FP:", total_FP)
print("Total FN:", total_FN)


In [ ]:
# ALL METRIC 
import math

prec= total_TP/(total_TP+total_FP)
rec = total_TP/(total_TP+total_FN)
f1=2*prec*rec/(prec+rec)

# Print the metrics
print("Precision:", round(prec,3))
print("Recall:", round(rec,3))
print("F1-score:", round(f1,3))

tanpa layer lstm tambahan:

Precision: 0.9708141321044547
Recall: 0.9210784779647055
F1-score: 0.9452925588125315